In [223]:
import base64
from base64 import b64encode, b64decode
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

def valid_pkcs7(plain):
    return 1 <= plain[-1] <= 16 and plain[-plain[-1]:] == bytes([plain[-1]]) * plain[-1]

def unpkcs7(plain):
    if 1 <= plain[-1] <= 16 and plain[-plain[-1]:] == bytes([plain[-1]]) * plain[-1]:
        return plain[:-plain[-1]]
    else:
        raise ValueError("Bad padding")

B = lambda s: [s[i:i+16] for i in range(0, len(s), 16)]

def change_byte(b, pos, B):
    pos = pos % len(B)
    return B[:pos] + b + B[pos+1:]

def fixed_xor(a, b):
    return bytes([_a ^ _b for _a, _b in zip(a, b)])

In [3]:
K = np.random.bytes(16)

In [4]:
def server_cipher(key=K):

    P = b"""MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=
    MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=
    MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==
    MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==
    MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl
    MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==
    MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==
    MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=
    MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=
    MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93""".splitlines()
    P = [base64.decodebytes(p) for p in P]

    padder = padding.PKCS7(128).padder()
    prefix = np.random.choice(P)
    plain = padder.update(prefix) + padder.finalize()

    iv = np.random.bytes(16)
    aes = Cipher(algorithms.AES128(key), modes.CBC(iv)).encryptor()
    ciphertxt = aes.update(plain) + aes.finalize()

    return iv, ciphertxt

In [5]:
def padding_oracle(iv, ciphertxt, key=K):
    aes = Cipher(algorithms.AES128(key), modes.CBC(iv)).decryptor()
    plain = aes.update(ciphertxt) + aes.finalize()
    return valid_pkcs7(plain)

In [6]:
iv, ciphertxt = server_cipher()

In [211]:
def break_P2(C0, C1, C2, padding_oracle):

    P2 = b''
    C1_blkp = C1

    for i in range(256):
        C1_prime = change_byte(bytes([i]), -1, C1)
        if padding_oracle(C0, C1_prime + C2):
            C1_second = change_byte(bytes([(C1_prime[-2] >> 1) ^ C1_prime[-2]]), -2, C1_prime)
            if padding_oracle(C0, C1_second + C2):
                pad_byte = bytes([C1_prime[-1] ^ 0x01 ^ C1[-1]])
                break

    P2 = pad_byte + P2

    for i in range(1, 16):

        C1 = C1_blkp

        for j in range(1, i+1):
            C1 = change_byte(bytes([P2[-j] ^ C1[-j] ^ (i+1)]), -j, C1)

        for k in range(256):
            C1_prime = change_byte(bytes([k]), -(i+1), C1)
            if padding_oracle(C0, C1_prime + C2):
                pad_byte = bytes([C1_prime[-(i+1)] ^ (i+1) ^ C1[-(i+1)]])
                break

        P2 = pad_byte + P2

        print(f"\rDecrypting block: {P2}", end="")
    
    print("")
    
    return P2


In [8]:
C = B(ciphertxt)
break_P2(np.random.bytes(16), iv, C[0], padding_oracle)

Decrypting block: b'000005I go crazy'


b'000005I go crazy'

In [4]:
def padding_oracle_attack(iv, ciphertxt, padding_oracle):

    blks = [b'\x00' * 16, iv] + B(ciphertxt)

    return unpkcs7(b''.join([break_P2(blks[i], blks[i+1], blks[i+2], padding_oracle) for i in range(len(blks) - 3 + 1)]))

In [101]:
padding_oracle_attack(iv, ciphertxt, padding_oracle)

b"000004Burning 'em, if you ain't quick and nimble"

In [115]:
%%time
padding_oracle_attack(*server_cipher(), padding_oracle)

Decrypting block: b"000008ollin' in "
Decrypting block: b'my five point oh'
Decrypting block: b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
CPU times: user 1.44 s, sys: 1.51 s, total: 2.95 s
Wall time: 3.16 s


b"000008ollin' in my five point oh"

### Mock vulnerable webapp

In [20]:
import json

In [435]:
class Pastebin:

    db = {}

    def __init__(self):
        self.staticKey = np.random.bytes(16)
    
    def post(self, title, data):
        id = len(self.db)
        post_key = np.random.bytes(16)
        cipher = self.encrypt_json_data(post_key, {'title': title, 'data': data})
        self.db[id] = cipher
        post_cipher = self.encrypt_json_data(self.staticKey, {'id': id, 'key': base64.b64encode(post_key).decode('utf-8')})
        return post_cipher
    
    def encrypt_json_data(self, key, json_data):
        """base64 of a cipher who's plain text is base64 json string"""
        plain = json.dumps(json_data)
        plainb64 = base64.b64encode(plain.encode('utf-8'))
        padder = padding.PKCS7(128).padder()
        padded_plain = padder.update(plainb64) + padder.finalize()
        iv = np.random.bytes(16)
        aes = Cipher(algorithms.AES128(key), modes.CBC(iv)).encryptor()
        ciphertxt = aes.update(padded_plain) + aes.finalize()
        cipher = iv + ciphertxt
        return base64.b64encode(cipher)
    
    def decrypt_json_data(self, key, cipher):
        cipher = base64.b64decode(cipher)
        iv, ciphertxt = cipher[:16], cipher[16:]
        aes = Cipher(algorithms.AES128(key), modes.CBC(iv)).decryptor()
        plain = aes.update(ciphertxt) + aes.finalize()
        unpadder = padding.PKCS7(128).unpadder()
        unpadded_plain = unpadder.update(plain) + unpadder.finalize()
        return json.loads(base64.decodebytes(unpadded_plain))
    
    def get(self, post):
        post = self.decrypt_json_data(self.staticKey, post)
        if 'admin' in post:
            return f'Hi admin, your static key is {self.staticKey}'
        cipher_bin = self.db[post['id']]
        plain_bin = self.decrypt_json_data(base64.b64decode(post['key']), cipher_bin)
        return plain_bin

In [436]:
p = Pastebin()
p1 = p.post('first', 'bin')
p2 = p.post('second', 'bin2')
p3 = p.post('third', 'bin3')
print(p1, p2, p3)
print(p.get(p1), p.get(p2), p.get(p3))

b'I3zg/XnGBV9NyeYA9NIaPPjEur9sRUYn4AvsWoA1DMDwniyxV1trYUicFXYJz80wMfjzsmjkW/19DOAWkBCIb4U2zSzTtyMYWpeOWumnrYA=' b'ZIIB7nA5IZ2kY0F3OPaH9mD81aK6JxrkcKl0zOntfl0D/8PhwI22CjfSINBITov/o57vsQhBhusaPUwnrw0q6g5qjzyG3xf+lwrpeii4GRc=' b'OdVFhUaRoUu06EzcE+ACjDIB0quE/cxoKIJiD9gO5pGXqwq/gfiOoMz1dxHQpLzkbv5x8T9QrV7v68Wn+yY/EtHyM6waRFEX76i0BzndQyA='
{'title': 'first', 'data': 'bin'} {'title': 'second', 'data': 'bin2'} {'title': 'third', 'data': 'bin3'}


In [437]:
import traceback

def padding_oracle(cipher):
    try:
        p.get(cipher)
        return True
    except Exception as e:
        return 'Invalid padding bytes' not in str(e)

In [438]:
padding_oracle(base64.b64encode(b'\x00' * 16 + b'\xaa' + b'\x00' * 15))

False

In [439]:
print(padding_oracle(p1))

True


In [372]:
def break_P2(C0, C1, C2, padding_oracle):

    P2 = b''
    C1_blkp = C1

    for i in range(256):
        C1_prime = change_byte(bytes([i]), -1, C1)
        if padding_oracle(b64encode(C0 + C1_prime + C2)):
            C1_second = change_byte(bytes([(C1_prime[-2] >> 1) ^ C1_prime[-2]]), -2, C1_prime)
            if padding_oracle(b64encode(C0 + C1_second + C2)):
                pad_byte = bytes([C1_prime[-1] ^ 0x01 ^ C1[-1]])
                break

    P2 = pad_byte + P2

    for i in range(1, 16):

        C1 = C1_blkp

        for j in range(1, i+1):
            C1 = change_byte(bytes([P2[-j] ^ C1[-j] ^ (i+1)]), -j, C1)

        for k in range(256):
            C1_prime = change_byte(bytes([k]), -(i+1), C1)
            if padding_oracle(b64encode(C0 + C1_prime + C2)):
                pad_byte = bytes([C1_prime[-(i+1)] ^ (i+1) ^ C1[-(i+1)]])
                break

        P2 = pad_byte + P2
    
    return P2


In [371]:
def padding_oracle_attack(cipher, padding_oracle):
    iv, ciphertxt = cipher[:16], cipher[16:]
    blks = [b'\x00' * 16, iv] + B(ciphertxt)
    return b''.join([break_P2(blks[i], blks[i+1], blks[i+2], padding_oracle) for i in range(len(blks) - 3 + 1)])

In [440]:
p1b = b64decode(p1)
b64decode(break_P2(b'\x00' * 16, p1b[:16], p1b[16:32], padding_oracle))

b'{"id": 0, "k'

In [441]:
def pad(msg):
    padder = padding.PKCS7(128).padder()
    return padder.update(msg) + padder.finalize()

In [442]:
IV, C0, C1, C2, C3 = B(p1b)
IV, C0, C1, C2, C3

(b'#|\xe0\xfdy\xc6\x05_M\xc9\xe6\x00\xf4\xd2\x1a<',
 b"\xf8\xc4\xba\xbflEF'\xe0\x0b\xecZ\x805\x0c\xc0",
 b'\xf0\x9e,\xb1W[kaH\x9c\x15v\t\xcf\xcd0',
 b'1\xf8\xf3\xb2h\xe4[\xfd}\x0c\xe0\x16\x90\x10\x88o',
 b'\x856\xcd,\xd3\xb7#\x18Z\x97\x8eZ\xe9\xa7\xad\x80')

In [443]:
P0, P1, P2, P3 = B(padding_oracle_attack(IV + C0 + C1 + C2 + C3, padding_oracle))
P0, P1, P2, P3

(b'eyJpZCI6IDAsICJr',
 b'ZXkiOiAiRnFFeHhJ',
 b'UlZSYnBkYllWK1da',
 b'ZUExZz09In0=\x04\x04\x04\x04')

In [444]:
forged_msg = '{"id": 1, "admin": true}'.ljust(len(b64decode(plain)), '\n').encode('utf-8')
print(forged_msg, len(forged_msg))
print(b64decode(plain), len(b64decode(plain)))

b'{"id": 1, "admin": true}\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n' 44
b'{"id": 3, "key": "d7nBRaab8dcOF5gfvS8x8Q=="}' 44


In [445]:
F0, F1, F2, F3 = B(pad(b64encode(forged_msg)))
F0, F1, F2, F3

(b'eyJpZCI6IDEsICJh',
 b'ZG1pbiI6IHRydWV9',
 b'CgoKCgoKCgoKCgoK',
 b'CgoKCgoKCgo=\x04\x04\x04\x04')

In [446]:
D3 = fixed_xor(C2, P3)

In [447]:
C2_prime = fixed_xor(D3, F3)

In [448]:
res = B(padding_oracle_attack(IV + C0 + C1 + C2_prime + C3, padding_oracle))
assert P0 == res[0]
assert P1 == res[1]
assert F3 == res[3]
P2_prime = res[2]
res

[b'eyJpZCI6IDAsICJr',
 b'ZXkiOiAiRnFFeHhJ',
 b'\xef\xb0u7S0\xe8\xb1\xed\xb6\xfch\xab\xd4s\x99',
 b'CgoKCgoKCgo=\x04\x04\x04\x04']

In [449]:
D2_prime = fixed_xor(C1, P2_prime)

In [450]:
C1_prime = fixed_xor(D2_prime, F2)

In [451]:
res = B(padding_oracle_attack(IV + C0 + C1_prime + C2_prime + C3, padding_oracle))
assert P0 == res[0]
P1_prime = res[1]
assert F2 == res[2]
assert F3 == res[3]
res

[b'eyJpZCI6IDAsICJr',
 b'"\xcf\xcc\xcf\xa8\xd8\xbe$\xf2\x00\x9a{p\xa1$U',
 b'CgoKCgoKCgoKCgoK',
 b'CgoKCgoKCgo=\x04\x04\x04\x04']

In [452]:
D1_prime = fixed_xor(C0, P1_prime)
C0_prime = fixed_xor(D1_prime, F1)
res = B(padding_oracle_attack(IV + C0_prime + C1_prime + C2_prime + C3, padding_oracle))
P0_prime = res[0]
assert F1 == res[1]
assert F2 == res[2]
assert F3 == res[3]
res

[b'\x13\xd4C\x1a\x1a\x13H;\xae\xf4\xd3\xe14\xe6:\xcd',
 b'ZG1pbiI6IHRydWV9',
 b'CgoKCgoKCgoKCgoK',
 b'CgoKCgoKCgo=\x04\x04\x04\x04']

In [453]:
D0_prime = fixed_xor(IV, P0_prime)
IV_prime = fixed_xor(D0_prime, F0)
res = B(padding_oracle_attack(IV_prime + C0_prime + C1_prime + C2_prime + C3, padding_oracle))
assert F0 == res[0]
assert F1 == res[1]
assert F2 == res[2]
assert F3 == res[3]
res

[b'eyJpZCI6IDEsICJh',
 b'ZG1pbiI6IHRydWV9',
 b'CgoKCgoKCgoKCgoK',
 b'CgoKCgoKCgo=\x04\x04\x04\x04']

In [454]:
p.get(b64encode(IV_prime + C0_prime + C1_prime + C2_prime + C3))

'Hi admin, your static key is b\'\\x9e\\x7f"\\xe1\\xab\\xef\\xdd\\xa9&8\\x90\\x1eR\\xb6\\xbc\\xab\''

⚠️ No need for a padding attack on the whole cipher, only the relevant block